In [68]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [69]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [70]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [71]:
dfAdmDiff = dfAdm.diff().iloc[1:]
plotDates = curDates[1:]

In [72]:
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [73]:
# Collect the cases, tests and positive percentage from all the "cases by age" files

ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        
        curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))
        
        
        dfCase.loc[latestDate] = curdf['Antal_bekræftede_COVID-19'].values
        dfTest.loc[latestDate] = curdf['Antal_testede'].values
        dfPosP.loc[latestDate] = curdf['Procent_positive'].values

allDates = dfCase.index


In [74]:
# # Collect the cases, tests and positive percentage from all the "cases by age" files

# ssidatapath = "ssi_data"
# rootdir = os.getcwd() +"/" + ssidatapath

# dfAdmNew = pd.DataFrame(columns=['Indlagt'])

# for subdir, dirs, files in os.walk(rootdir):
#     if not len(files) == 0:
#         latestdir = subdir
#         latestDate = pd.to_datetime(subdir[-10:])
#         curdf = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
        
#         try: 
#             curdf['Antal_nyindlagte'] = pd.to_numeric(curdf['Antal_nyindlagte'].astype(str).apply(lambda x: x.replace('.','')))
#             dfAdmNew.loc[latestDate,'Indlagt'] = curdf['Antal_nyindlagte'].values[-1]
#         except:
#             curdf['Total'] = pd.to_numeric(curdf['Total'].astype(str).apply(lambda x: x.replace('.','')))
#             dfAdmNew.loc[latestDate,'Indlagt'] = curdf['Total'].values[-1]
        
        

# # allDates = dfCase.index
# # dfAdmNew

dfAdmNew = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
dfAdmNew['Dato'] = pd.to_datetime(dfAdmNew['Dato'])
dfAdmNew['Total'] = pd.to_numeric(dfAdmNew['Total'])
dfAdmNew.tail()


dfDea = pd.read_csv(latestdir+'/Deaths_over_time.csv',delimiter = ';',dtype=str)
dfDea = dfDea.iloc[:-1,:]
dfDea['Dato'] = pd.to_datetime(dfDea['Dato'])
dfDea['Antal_døde'] = pd.to_numeric(dfDea['Antal_døde'])
dfDea.tail()

,Dato,Antal_døde
592,2021-10-24,0
593,2021-10-25,4
594,2021-10-26,3
595,2021-10-27,1
596,2021-10-28,0


In [75]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfTestDiff = dfTest.diff().iloc[1:]
dfPosPDiff = dfPosP.diff().iloc[1:]
plotDatesCase = allDates[1:]

In [76]:
fig,ax1 = plt.subplots(1,1)

howManyToUse = len(dfCaseDiff.columns)-4
cmap = plt.cm.get_cmap('Dark2',howManyToUse)
for k in range(0,howManyToUse):
    curColName = dfCaseDiff.columns[k]
    ax1.plot(plotDatesCase,dfCaseDiff[curColName],'.-',linewidth=0.4,label=curColName,color=cmap(k))
    # meanWidth = 7
    # ax1.plot(rnTime(plotDatesCase,meanWidth),rnMean(dfCaseDiff[curColName],meanWidth),label=curColName,color=cmap(k))


# testMonday = np.datetime64('2021-05-17')
testMonday = np.datetime64('2021-03-01')
numMondays = 50
for m in range(0,numMondays):
    ax1.bar(testMonday+np.timedelta64(m*7,'D'),1000,color='grey')

ax1.legend()

ax1.set_ylim(bottom=0,top=1000)
# ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-05-05')])
ax1.set_xlim(left=np.datetime64('2021-05-01'))
ax1.set_xlim(left=np.datetime64('2021-08-01'))
ax1.set_xlim(right=np.datetime64(date.today()))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
# Compare years
allDatesDiff = allDates[1:]

fig,ax1 = plt.subplots(1,1)
prevYearIndex = (allDatesDiff < np.datetime64('2021-01-01'))
prevYearDates = allDatesDiff[prevYearIndex]
prevYearSum = dfCaseDiff[prevYearIndex]['I alt']

thisYearIndex = (allDatesDiff >= np.datetime64('2021-01-01'))
thisYearDates = allDatesDiff[thisYearIndex]
thisYearSum = dfCaseDiff[thisYearIndex]['I alt']

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum)
ax1.plot(thisYearDates,thisYearSum)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:

curdf = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
# curdf

In [79]:
curdf = curdf.iloc[:-2]
# curdf['NewPositive'] = pd.to_numeric(curdf['NewPositive'])
curdf['NewPositive'] = pd.to_numeric(curdf['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
curdf['Tested'] = pd.to_numeric(curdf['Tested'].astype(str).apply(lambda x: x.replace('.','')))
curdf['PosPct'] = pd.to_numeric(curdf['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))

testDates =  pd.to_datetime(curdf.Date,format='%Y-%m-%d')
# curdf

In [80]:
# plt.figure()
# curdf

In [81]:

fig,ax1 = plt.subplots(1,1)
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,label='2020')
ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],label='2021')
# ax1.plot(thisYearDates,thisYearSum,label='2021')

ax1.legend()
ax1.grid()
ax1.set_ylabel('Antal tilfælde')
ax1.set_ylim(bottom=0)
ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))

plt.tight_layout()
plt.savefig('figs/Compare2020and2021')


ax1.set_ylim(top=1500)
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-11-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021Zoom')

ax1.set_ylim(top=3000)
ax1.set_xlim([np.datetime64('2021-06-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021Zoom2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']
prevYearTest = curdf[prevYearIndex]['Tested']
# prevYearPos = curdf[prevYearIndex]['PosPct']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']
thisYearTest = curdf[thisYearIndex]['Tested']
# thisYearPos = curdf[thisYearIndex]['PosPct']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')
ax1.plot(thisYearDates.values,thisYearSum.values,'r.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values,meanWidth),rnMean(thisYearSum.values,meanWidth),'r',label='2021')


ax2.plot(prevYearDates+np.timedelta64(365,'D'),prevYearTest,'b.:',linewidth=0.5,markersize=4)
ax2.plot(thisYearDates.values[:-1],thisYearTest.values[:-1],'r.:',linewidth=0.5,markersize=4)
ax2.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearTest,meanWidth),'b',label='2020')
ax2.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearTest.values[:-1],meanWidth),'r',label='2021') 


ax3.plot(prevYearDates+np.timedelta64(365,'D'),prevYearPos,'b.:',linewidth=0.5,markersize=4)
ax3.plot(thisYearDates.values[:-1],thisYearPos.values[:-1],'r.:',linewidth=0.5,markersize=4)
ax3.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearPos,meanWidth),'b',label='2020')
ax3.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearPos.values[:-1],meanWidth),'r',label='2021')


xTicks = np.arange(np.datetime64('2021-01'),np.datetime64('2022-01'))
ax1.set_xticks(xTicks)
# plt.xticks(rotation=90)

ax1.legend()
ax1.grid()
ax2.grid()
ax3.grid()
ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Antal test')
ax3.set_ylabel('Positiv procent [%]')
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)
ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All')

xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-11'),np.timedelta64(14,'D'))
ax1.set_xticks(xTicks)

ax3.set_ylim(top=3)

ax1.set_ylim(top=2500)
ax1.set_xlim([np.datetime64('2021-04-01'),np.datetime64('2021-12-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All_Zoom')

# xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-08'),np.timedelta64(14,'D'))
# ax1.set_xticks(xTicks)

# ax3.set_ylim(top=1)

# ax1.set_ylim(top=1200)
ax1.set_xlim([np.datetime64('2021-06-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All_Zoom_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:

fig,ax1 = plt.subplots(1,1)

# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']
prevYearTest = curdf[prevYearIndex]['Tested']
# prevYearPos = curdf[prevYearIndex]['PosPct']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']
thisYearTest = curdf[thisYearIndex]['Tested']
# thisYearPos = curdf[thisYearIndex]['PosPct']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')
ax1.plot(thisYearDates.values,thisYearSum.values,'r.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values,meanWidth),rnMean(thisYearSum.values,meanWidth),'r',label='2021')










xTicks = np.arange(np.datetime64('2021-01'),np.datetime64('2022-01'))
ax1.set_xticks(xTicks)
# plt.xticks(rotation=90)

ax1.legend()
ax1.grid()

ax1.set_ylabel('Antal tilfælde')

ax1.set_ylim(bottom=0)

ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.tight_layout()
# plt.savefig('figs/Compare2020and2021_All')

xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-11'),np.timedelta64(14,'D'))
ax1.set_xticks(xTicks)


ax1.set_ylim(top=1600)
ax1.set_xlim([np.datetime64('2021-04-01'),np.datetime64('2021-12-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
# plt.savefig('figs/Compare2020and2021_All_Zoom')

# xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-08'),np.timedelta64(14,'D'))
# ax1.set_xticks(xTicks)


ax1.set_ylim(top=4500)
ax1.set_xlim([np.datetime64('2021-08-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_Cases_Fall')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:

# ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')

dfToSave2020 = pd.DataFrame()
dfToSave2020['Dato'] = prevYearDates+np.timedelta64(365,'D')
dfToSave2020['Cases'] = prevYearSum
# dfToSave2020['Cases2021'] = thisYearSum.values[:-1]
# dfToSave2020

dfToSave2021 = pd.DataFrame()
dfToSave2021['Dato'] = thisYearDates.values
dfToSave2021['Cases'] =  thisYearSum.values
# dfToSave2020['Cases2021'] = thisYearSum.values[:-1]


dfToSaveRM2020 = pd.DataFrame()
# rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth)
dfToSaveRM2020['Dato'] = rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth)
dfToSaveRM2020['Cases'] = rnMean(prevYearSum,meanWidth)
# dfToSaveRM2020['Cases2021'] = rnMean(thisYearSum.values[:-1],meanWidth)
# asdfasdf
# dfToSaveRM2020

dfToSaveRM2021 = pd.DataFrame()
dfToSaveRM2021['Dato'] = rnTime(thisYearDates.values,meanWidth)
dfToSaveRM2021['Cases'] = rnMean(thisYearSum.values,meanWidth)

# dfToSave2020.to_excel('Cases2020and2021')
# dfToSave2021

In [85]:
dfToSave2020.to_excel('Cases2020.xlsx',index=False)
dfToSaveRM2020.to_excel('Cases2020RunningMean.xlsx',index=False)
dfToSave2021.to_excel('Cases2021.xlsx',index=False)
dfToSaveRM2021.to_excel('Cases2021RunningMean.xlsx',index=False)

# dfToSave2020.to_excel('Cases2020.xlsx',index=False)
# dfToSaveRM2020.to_excel('Cases2020RunningMean.xlsx',index=False)

In [86]:
fig,ax1 = plt.subplots()

ax1.plot(dfToSave2020.Dato,dfToSave2020.Cases,'.:')
ax1.plot(dfToSave2021.Dato,dfToSave2021.Cases,'.:')
ax1.plot(dfToSaveRM2020.Dato,dfToSaveRM2020.Cases)
ax1.plot(dfToSaveRM2021.Dato,dfToSaveRM2021.Cases)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:

# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
fig,ax1 = plt.subplots(figsize=(6,10))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']
prevYearTest = curdf[prevYearIndex]['Tested']
# prevYearPos = curdf[prevYearIndex]['PosPct']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']
thisYearTest = curdf[thisYearIndex]['Tested']
# thisYearPos = curdf[thisYearIndex]['PosPct']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7
ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'m.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'k',label='2021')

curXTicks = [
    np.datetime64('2021-08-01'),
    np.datetime64('2021-08-15'),
    np.datetime64('2021-09-01'),
    np.datetime64('2021-09-15'),
    np.datetime64('2021-10-01'),
    np.datetime64('2021-10-15'),
    np.datetime64('2021-11-01'),
    np.datetime64('2021-11-15'),
]
ax1.set_xticks(curXTicks)
plt.xticks(fontsize=15)

ax1.set_xlim([np.datetime64('2021-08-10'),np.datetime64('2021-11-17')])
ax1.set_ylim([0,3400])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.grid()

# ax1.savefig('figs/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:

fig,ax1 = plt.subplots(1,1)

testDates = dfAdmNew.Dato

# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = dfAdmNew[prevYearIndex]['Total']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = dfAdmNew[thisYearIndex]['Total']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')
ax1.plot(thisYearDates.values,thisYearSum.values,'r.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values,meanWidth),rnMean(thisYearSum.values,meanWidth),'r',label='2021')


xTicks = np.arange(np.datetime64('2021-01'),np.datetime64('2022-01'))
ax1.set_xticks(xTicks)
# plt.xticks(rotation=90)

ax1.legend()
ax1.grid()

ax1.set_ylabel('Antal nyindlæggelser')

ax1.set_ylim(bottom=0)

xTicks = np.arange(np.datetime64('2021-08-01'),np.datetime64('2022-01-11'),np.timedelta64(14,'D'))
xTicks = np.arange(np.datetime64('2021-08'),np.datetime64('2022-03'))
ax1.set_xticks(xTicks)


curXTicks = [
    np.datetime64('2021-08-01'),
    np.datetime64('2021-08-15'),
    np.datetime64('2021-09-01'),
    np.datetime64('2021-09-15'),
    np.datetime64('2021-10-01'),
    np.datetime64('2021-10-15'),
    np.datetime64('2021-11-01'),
    np.datetime64('2021-11-15'),
    np.datetime64('2021-12-01'),
    np.datetime64('2021-12-15'),
    np.datetime64('2022-01-01'),
    np.datetime64('2022-01-15'),
]
ax1.set_xticks(curXTicks)

# ax1.set_ylim(top=250)
ax1.set_xlim([np.datetime64('2021-08-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_Admitted_Fall')

# ax1.set_ylim(top=250)
ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.set_xticks(np.arange(np.datetime64('2021-01'),np.datetime64('2022-02')))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_Admitted_WholeYear')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:

dfToSave2020 = pd.DataFrame()
dfToSave2020['Dato'] = prevYearDates+np.timedelta64(365,'D')
dfToSave2020['Indlagte'] = prevYearSum
# dfToSave2020['Indlagte2021'] = thisYearSum.values[:-1]
# dfToSave2020

dfToSave2021 = pd.DataFrame()
dfToSave2021['Dato'] = thisYearDates.values
dfToSave2021['Indlagte'] =  thisYearSum.values
# dfToSave2020['Indlagte2021'] = thisYearSum.values[:-1]


dfToSaveRM2020 = pd.DataFrame()
# rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth)
dfToSaveRM2020['Dato'] = rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth)
dfToSaveRM2020['Indlagte'] = rnMean(prevYearSum,meanWidth)
# dfToSaveRM2020['Indlagte2021'] = rnMean(thisYearSum.values[:-1],meanWidth)
# asdfasdf
# dfToSaveRM2020

dfToSaveRM2021 = pd.DataFrame()
dfToSaveRM2021['Dato'] = rnTime(thisYearDates.values,meanWidth)
dfToSaveRM2021['Indlagte'] = rnMean(thisYearSum.values,meanWidth)

# dfToSave2020.to_excel('Cases2020and2021')
# dfToSave2021

In [90]:
dfToSave2020.to_excel('Indlagte2020.xlsx',index=False)
dfToSaveRM2020.to_excel('Indlagte2020RunningMean.xlsx',index=False)
dfToSave2021.to_excel('Indlagte2021.xlsx',index=False)
dfToSaveRM2021.to_excel('Indlagte2021RunningMean.xlsx',index=False)


In [91]:
fig,ax1 = plt.subplots()

ax1.plot(dfToSave2020.Dato,dfToSave2020.Indlagte,'.:')
ax1.plot(dfToSave2021.Dato,dfToSave2021.Indlagte,'.:')
ax1.plot(dfToSaveRM2020.Dato,dfToSaveRM2020.Indlagte)
ax1.plot(dfToSaveRM2021.Dato,dfToSaveRM2021.Indlagte)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:

fig,ax1 = plt.subplots(1,1)

testDates = dfDea.Dato

# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = dfDea[prevYearIndex]['Antal_døde']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = dfDea[thisYearIndex]['Antal_døde']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')
ax1.plot(thisYearDates.values,thisYearSum.values,'r.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values,meanWidth),rnMean(thisYearSum.values,meanWidth),'r',label='2021')


xTicks = np.arange(np.datetime64('2021-01'),np.datetime64('2022-01'))
ax1.set_xticks(xTicks)
# plt.xticks(rotation=90)

ax1.legend()
ax1.grid()

ax1.set_ylabel('Antal dødsfald')

ax1.set_ylim(bottom=0)

xTicks = np.arange(np.datetime64('2021-08-01'),np.datetime64('2022-01-11'),np.timedelta64(14,'D'))
xTicks = np.arange(np.datetime64('2021-08'),np.datetime64('2022-03'))
ax1.set_xticks(xTicks)


curXTicks = [
    np.datetime64('2021-08-01'),
    np.datetime64('2021-08-15'),
    np.datetime64('2021-09-01'),
    np.datetime64('2021-09-15'),
    np.datetime64('2021-10-01'),
    np.datetime64('2021-10-15'),
    np.datetime64('2021-11-01'),
    np.datetime64('2021-11-15'),
    np.datetime64('2021-12-01'),
    np.datetime64('2021-12-15'),
    np.datetime64('2022-01-01'),
    np.datetime64('2022-01-15'),
]
ax1.set_xticks(curXTicks)

# ax1.set_ylim(top=250)
ax1.set_xlim([np.datetime64('2021-08-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_Deaths_Fall')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
dfDea

,Dato,Antal_døde
0,2020-03-11,1
1,2020-03-12,0
2,2020-03-13,0
3,2020-03-14,1
4,2020-03-15,2
5,2020-03-16,1
6,2020-03-17,1
7,2020-03-18,4
8,2020-03-19,5
9,2020-03-20,4


In [94]:

dfToSave2020 = pd.DataFrame()
dfToSave2020['Dato'] = prevYearDates+np.timedelta64(365,'D')
dfToSave2020['Deaths'] = prevYearSum
# dfToSave2020['Deaths2021'] = thisYearSum.values[:-1]
# dfToSave2020

dfToSave2021 = pd.DataFrame()
dfToSave2021['Dato'] = thisYearDates.values
dfToSave2021['Deaths'] =  thisYearSum.values
# dfToSave2020['Deaths2021'] = thisYearSum.values[:-1]


dfToSaveRM2020 = pd.DataFrame()
# rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth)
dfToSaveRM2020['Dato'] = rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth)
dfToSaveRM2020['Deaths'] = rnMean(prevYearSum,meanWidth)
# dfToSaveRM2020['Deaths2021'] = rnMean(thisYearSum.values[:-1],meanWidth)
# asdfasdf
# dfToSaveRM2020

dfToSaveRM2021 = pd.DataFrame()
dfToSaveRM2021['Dato'] = rnTime(thisYearDates.values,meanWidth)
dfToSaveRM2021['Deaths'] = rnMean(thisYearSum.values,meanWidth)

# dfToSave2020.to_excel('Cases2020and2021')
# dfToSave2021

In [95]:
dfToSave2020.to_excel('Deaths2020.xlsx',index=False)
dfToSaveRM2020.to_excel('Deaths2020RunningMean.xlsx',index=False)
dfToSave2021.to_excel('Deaths2021.xlsx',index=False)
dfToSaveRM2021.to_excel('Deaths2021RunningMean.xlsx',index=False)


In [96]:
fig,ax1 = plt.subplots()

ax1.plot(dfToSave2020.Dato,dfToSave2020.Deaths,'.:')
ax1.plot(dfToSave2021.Dato,dfToSave2021.Deaths,'.:')
ax1.plot(dfToSaveRM2020.Dato,dfToSaveRM2020.Deaths)
ax1.plot(dfToSaveRM2021.Dato,dfToSaveRM2021.Deaths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:

# # fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
# fig,ax1 = plt.subplots(figsize=(6,10))
# prevYearIndex = (testDates < np.datetime64('2021-01-01'))
# prevYearDates = testDates[prevYearIndex]
# prevYearSum = curdf[prevYearIndex]['NewPositive']
# prevYearTest = curdf[prevYearIndex]['Tested']
# # prevYearPos = curdf[prevYearIndex]['PosPct']
# prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

# thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
# thisYearDates = testDates[thisYearIndex]
# thisYearSum = curdf[thisYearIndex]['NewPositive']
# thisYearTest = curdf[thisYearIndex]['Tested']
# # thisYearPos = curdf[thisYearIndex]['PosPct']
# thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

# meanWidth=7
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'m.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'k',label='2021')

# curXTicks = [
#     np.datetime64('2021-08-01'),
#     np.datetime64('2021-08-15'),
#     np.datetime64('2021-09-01'),
#     np.datetime64('2021-09-15'),
#     np.datetime64('2021-10-01'),
#     np.datetime64('2021-10-15'),
#     np.datetime64('2021-11-01'),
#     np.datetime64('2021-11-15'),
# ]
# ax1.set_xticks(curXTicks)
# plt.xticks(fontsize=15)

# ax1.set_xlim([np.datetime64('2021-08-10'),np.datetime64('2021-11-17')])
# ax1.set_ylim([0,3400])

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
# ax1.grid()



In [98]:
# fig,ax1 = plt.subplots(1,1)
# meanWidth=7

# allCase = curdf['NewPositive']
# meanDates = rnTime(testDates,meanWidth)
# allCaseMean = rnMean(allCase,meanWidth)

# # ax1.plot(testDates,np.log(allCase),'k.:',linewidth=0.5,markersize=3)
# # ax1.plot(rnTime(testDates,meanWidth),rnMean(np.log(allCase),meanWidth),'k')



# # numPred = 30
# # curT = np.arange(0,numPred)
# # predDays = meanDates[-numPred:]
# # curI0 = allCaseMean[-numPred]
# iniPred = 60
# endPred = iniPred - 14
# curT = np.arange(0,iniPred-endPred)
# predDays = meanDates[-iniPred:-endPred]
# curI0 = allCaseMean[-iniPred]
# # predDays = testDates[-numPred:]
# # curI0 = allCase.values[-numPred]



# # allRts = np.arange(9,12,0.25)/10
# allRts = np.arange(10,13,0.5)/10

# # curRt = 1
# for curRt in allRts:
#     curModel = ( (curRt - 1) * curT / 4.7 ) + np.log(curI0)
#     ax1.plot(predDays,curModel,'--',label=str(curRt))



# ax1.plot(testDates,np.log(allCase),'k.:',linewidth=0.5,markersize=3)
# ax1.plot(rnTime(testDates,meanWidth),rnMean(np.log(allCase),meanWidth),'k')


# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

# # ax1.set_xlim(left=np.datetime64('2020-12-01'))
# # ax1.set_ylim(bottom=5)


# ax1.set_xlim(left=np.datetime64('2021-04-01'))
# ax1.set_ylim(bottom=4,top=7.5)

# ax1.legend()
# # ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
# # ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# # ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# # ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')

In [99]:
# 'Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte'
newfilename = "C:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData\\ssi_dashboard\\SSI_dashboard_2021-10-04\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv"

testdf = pd.read_csv(newfilename,delimiter = ';',dtype=str)
testdf['Testede pr. 100.000 borgere']  = pd.to_numeric(testdf['Testede pr. 100.000 borgere'])

testdf.Aldersgruppe.unique()

array(['0-2', '3-5', '6-11', '12-15', '16-19', '20-39', '40-64', '65-79',
       '80+'], dtype=object)

In [100]:


fig,ax1 = plt.subplots()

curAgeString = '12-15'
curagedf = testdf[testdf.Aldersgruppe == curAgeString].iloc[-20:]
ax1.plot(curagedf.Uge.str[-2:],curagedf['Testede pr. 100.000 borgere'],label=curAgeString)
curAgeString = '16-19'
curagedf = testdf[testdf.Aldersgruppe == curAgeString].iloc[-20:]
ax1.plot(curagedf.Uge.str[-2:],curagedf['Testede pr. 100.000 borgere'],label=curAgeString)
curAgeString = '20-39'
curagedf = testdf[testdf.Aldersgruppe == curAgeString].iloc[-20:]
ax1.plot(curagedf.Uge.str[-2:],curagedf['Testede pr. 100.000 borgere'],label=curAgeString)
curAgeString = '40-64'
curagedf = testdf[testdf.Aldersgruppe == curAgeString].iloc[-20:]
ax1.plot(curagedf.Uge.str[-2:],curagedf['Testede pr. 100.000 borgere'],label=curAgeString)

ax1.legend()

ax1.set_ylim(bottom=0)
# curagedf.Uge.str[-2:]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 31587.4)